In [1]:
from vqgan import VQGan
from vqgan import VQGan
from encoder import EncoderConfig
from decoder import DecoderConfig
from quantizer import QuantizerConfig
import torch.nn.functional as F
from lpips import LPIPS
from discriminator import Discriminator, DiscriminatorConfig
import torch
import torch.nn.functional as F
disc = Discriminator (DiscriminatorConfig)
import warnings
warnings.filterwarnings('always')


disc.to('cuda')
percept_disc = LPIPS().eval()
percept_disc.to('cuda')
x = torch.randn (1, 3, 256,256).to('cuda')
gan = VQGan (EncoderConfig, QuantizerConfig, DecoderConfig)
gan.to('cuda')

master_process = True
steps_per_epoch = 500
d_loss_factor = 1.0
grad_accum_steps = 4
lr = 6e-4

c:\Users\Zen\miniconda3\envs\tr\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Zen\miniconda3\envs\tr\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
e:\Research\vincentVanGogh\lpips.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-mod

In [2]:
def compute_loss (vqgan_model, discriminator, perceptual_distinguisher, x, perceptual_loss_factor, rec_loss_factor, current_step):
    print ("\n inside compute loss \n")
    reconstructed_iamges, encoding_indices, vq_loss = vqgan_model (x)
    print ("\n vqgan model forwded\n")
    # Discriminator Forward
    disc_real = discriminator (x)
    disc_generated = discriminator (reconstructed_iamges)
    print ("\nDISC forwarded\n")
    # VGG forward
    perceptual_loss = perceptual_distinguisher (x, reconstructed_iamges) # (B, 1, 1, 1)
    perceptual_loss = perceptual_loss.squeeze ( ) #(B)
    print ("\VGG FORWARDED\n")
    # formality none of the optimizers have access to vgg params but this way its explicit and efficient
    perceptual_loss = perceptual_loss.detach ( )

    reconstruction_loss = F.mse_loss (x, reconstructed_iamges, reduction='none') #(B, C, H, W)
    reconstruction_loss = reconstruction_loss.mean (dim=(1,2,3)) #(B)
    print ("\Rec loss computed\n")
    perceptual_recon_loss = rec_loss_factor * reconstruction_loss + perceptual_loss_factor * perceptual_loss
    perceptual_recon_loss = perceptual_recon_loss.mean()
    print ("\ncombined perceptual rec loss computed\n")

    # compute adversarial loss for generator if it's not zeroth epoch (VQGAN paper specs)
    is_zeroth_epoch = True if current_step < steps_per_epoch else False
    if is_zeroth_epoch:
        # don't calculate
        # Wasserstein loss, approximation of BCE (discriminator(generated), ones_like(generated))
        # TODO: Internalize throughly
        # good approximation of KLD between discriminator's predictions given generated inputs and ideal distribution of generated images being classified as real.
        # basically mean negative log likelihood is approximation of F.cross_entropy (discriminator(generated), ones_like(generated)) which is approximation of 
        # KLD between (discriminator(generated), ones_like(generated)) which is proportional to
        # BCE (discriminator(genrated), ones_like(generated))
        g_loss = 0 # replace by formula
        print ("\nlambda skipped\n")
    else:
        #calculate adversarial loss for generator
        g_loss = - torch.mean (disc_generated)
        lambda_factor = vqgan_model.compute_lambda (perceptual_recon_loss, g_loss)
        g_loss = lambda_factor * g_loss
        if master_process:
            print (f"\nGanLoss{g_loss:.6f} activated!\n")
        
    # VQ GAN LOSS
    # Net Generator Loss
    vq_gan_loss = perceptual_recon_loss + vq_loss + g_loss
    print ("VQGAN LOSS COMPUTED")
                    
    # Discriminator Loss : Hinge Loss. Push Logits for the catgeorical distribution that come out of discriminator 
    # to be more than +1 for real images
    # At the same time Push logits for categorical distribution that comes out of discriminator to be less than -1 for fake images

    # drive logits corresponding to real images from discriminator above zero
    # don't need to explicitly squeeze out the spurious depth dimension (1 channels) from discriminator output, mean will still be evaluated correctly
    d_loss_real= torch.mean(F.relu (1.0 - disc_real))
    # drive logits corresponding to fake(generate) images from discriminator below -1
    d_loss_fake = torch.mean(F.relu (1.0 + disc_generated))

    d_loss = d_loss_factor * 0.5 * (d_loss_real + d_loss_fake)
    return vq_gan_loss, d_loss

<>:12: DeprecationWarning: invalid escape sequence \V
<>:18: DeprecationWarning: invalid escape sequence \R
<>:12: DeprecationWarning: invalid escape sequence \V
<>:18: DeprecationWarning: invalid escape sequence \R
C:\Users\Zen\AppData\Local\Temp\ipykernel_29580\1499343910.py:12: DeprecationWarning: invalid escape sequence \V
  print ("\VGG FORWARDED\n")
C:\Users\Zen\AppData\Local\Temp\ipykernel_29580\1499343910.py:18: DeprecationWarning: invalid escape sequence \R
  print ("\Rec loss computed\n")


In [3]:
from torch.distributed import init_process_group, destroy_process_group
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import os

ddp = int (os.environ.get ('RANK', -1)) != -1 # is this a ddp run or not?
if ddp:
    assert torch.cuda.is_available(), f"CUDA is required for DDP"
    init_process_group (backend='nccl')
    ddp_rank = int (os.environ['RANK'])
    ddp_local_rank = int (os.environ['LOCAL_RANK'])
    ddp_world_size = int (os.environ['WORLD_SIZE'])
    device = f"cuda:{ddp_local_rank}"
    torch.cuda.set_device (device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc
else:
    # non ddp vanilla run
    ddp_rank = 0
    ddp_local_rank = 0
    ddp_world_size = 1
    master_process = True

    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda'
    elif hasattr (torch.backends, 'mps') and torch.backends.mps.is_available ():
        device = 'mps'
    print (f"using device: {device}")

ddp = int (os.environ.get ('RANK', -1)) != -1 # is this a ddp run or not?
if ddp:
    assert torch.cuda.is_available(), f"CUDA is required for DDP"
    init_process_group (backend='nccl')
    ddp_rank = int (os.environ['RANK'])
    ddp_local_rank = int (os.environ['LOCAL_RANK'])
    ddp_world_size = int (os.environ['WORLD_SIZE'])
    device = f"cuda:{ddp_local_rank}"
    torch.cuda.set_device (device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc
else:
    # non ddp vanilla run
    ddp_rank = 0
    ddp_local_rank = 0
    ddp_world_size = 1
    master_process = True

    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda'
    elif hasattr (torch.backends, 'mps') and torch.backends.mps.is_available ():
        device = 'mps'
    print (f"using device: {device}")

using device: cuda
using device: cuda


In [4]:
if ddp:
    # forward is unchanged, backward is mostly unchanged except there is overlap between computation and communication of gradients
    # while the backward pass is still going on, to average the gradients from all processes
    # we're tacking on this average as we will see in a bit
    gan = DDP (gan, device_ids=[ddp_local_rank])
    disc = DDP (disc, device_ids=[ddp_local_rank])
raw_gan = gan.module if ddp else gan
raw_discriminator = disc.module if ddp else disc

In [5]:
gan_opt = raw_gan.configure_optimizers (0.1, 6e-4, 'cuda')
gan.train()
gan_opt.zero_grad()
vqgan_loss_accum = 0.0

optimizer_configuration for VQGAN model:
 num decayed parameter tensors:50 with 104367168 parameters
num non-decayed parameter tensors:105 with 54467 parameters
using fused AdamW:True


In [6]:
for micro_step in range (grad_accum_steps):
    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
        vqgan_loss, dloss = compute_loss (gan, disc, percept_disc, x, 1.0, 1.0, 500)
    vqgan_loss = vqgan_loss / grad_accum_steps
    vqgan_loss_accum += vqgan_loss.detach()

    is_last_micro_step = (micro_step == (grad_accum_steps - 1))
    if ddp:
        # very last backward will have the grad_sync flag as True
        # for now this works, but not a good practice if pytorch takes the flag away
        # averages gradients
        gan.require_backward_grad_sync = is_last_micro_step
        disc.require_backward_grad_sync = is_last_micro_step
    
    vqgan_loss.backward (retain_graph=True)
    dloss.backward()

if ddp:
        # calculates average of loss_accum on all the ranks, and it deposits that average on all the ranks
        # all the ranks will contain loss_accum averaged up
        dist.all_reduce (vqgan_loss_accum, op= dist.ReduceOp.AVG)
vqgan_norm = torch.nn.utils.clip_grad_norm_ (gan.parameters(), 1.0)


 inside compute loss 


	ze computed


	applying quantizer 

on
torch.Size([1, 512, 16, 16])

	 rearranged ze for computing L2 with codebook


	 calculating distances for each of torch.Size([256, 1, 512]) with codebook vectors torch.Size([1024, 512])


	 calculated distances for each of torch.Size([256, 1, 512]) with codebook vectors torch.Size([1024, 512])


	quantizer applied


 vqgan model forwded


DISC forwarded

\VGG FORWARDED

\Rec loss computed


combined perceptual rec loss computed


GanLoss-1.209179 activated!

VQGAN LOSS COMPUTED

 inside compute loss 


	ze computed


	applying quantizer 

on
torch.Size([1, 512, 16, 16])

	 rearranged ze for computing L2 with codebook


	 calculating distances for each of torch.Size([256, 1, 512]) with codebook vectors torch.Size([1024, 512])


	 calculated distances for each of torch.Size([256, 1, 512]) with codebook vectors torch.Size([1024, 512])


	quantizer applied


 vqgan model forwded


DISC forwarded

\VGG FORWARDED

\Rec loss co

In [7]:
for param_group in gan_opt.param_groups:
        param_group['lr'] = lr

In [8]:
gan_opt.step()

In [9]:
for name, param in gan.named_parameters():
    if param.grad is not None:
        print(f"{name}: grad strides = {param.grad.stride()}, grad size = {param.grad.size()}")

encoder.initial_conv.weight: grad strides = (27, 9, 3, 1), grad size = torch.Size([128, 3, 3, 3])
encoder.initial_conv.bias: grad strides = (1,), grad size = torch.Size([128])
encoder.model.1.conv_projection.weight: grad strides = (2304, 9, 3, 1), grad size = torch.Size([256, 256, 3, 3])
encoder.model.1.conv_projection.bias: grad strides = (1,), grad size = torch.Size([256])
encoder.model.1.block.0.group_norm.weight: grad strides = (1,), grad size = torch.Size([128])
encoder.model.1.block.0.group_norm.bias: grad strides = (1,), grad size = torch.Size([128])
encoder.model.1.block.2.weight: grad strides = (1152, 9, 3, 1), grad size = torch.Size([256, 128, 3, 3])
encoder.model.1.block.2.bias: grad strides = (1,), grad size = torch.Size([256])
encoder.model.1.block.3.group_norm.weight: grad strides = (1,), grad size = torch.Size([256])
encoder.model.1.block.3.group_norm.bias: grad strides = (1,), grad size = torch.Size([256])
encoder.model.1.channel_up.weight: grad strides = (128, 1, 1, 1)

In [10]:
vqgan = VQGan (EncoderConfig, QuantizerConfig, DecoderConfig)
print(vqgan.module)

AttributeError: 'VQGan' object has no attribute 'module'

In [11]:

class DataloaderLite:
    # to load B images for each ddp process with T= HW tokens each has 3 Channels (RGB)
    def __init__(self, B, num_processes, process_rank, split, data_root):
        self.B = B
        self.num_processes = num_processes
        self.process_rank = process_rank
        assert split in {"train", "val"}, f"Invalid split specified at DataLoaderInstatntiation"
        assert os.path.exists (data_root)
        
        # get the shard filenames
        shards = os.listdir (data_root)
        shards = [s for s in shards if split in s]
        shards = [os.path.join(data_root, s) for s in shards]
        shards = sorted (shards)
        self.shards = shards
        assert len(shards) > 0, f"no shards found for split {split}"
        if master_process:
            print (f"found {len(shards)} shards for split {split}")
        # state init at shard zero
        self.reset ()
    
    def reset (self):
        self.current_shard = 0
        self.tokens = load_tokens (self.shards[self.current_shard])
        # B * T * process_rank
        # don't really need T here
        # B images go to one process
        self.current_position = self.process_rank * self.B
        self.current_epoch = 0

    def next_batch (self):
        B = self.B

        # inputs
        x = self.tokens [self.current_position : self.current_position + B] # (B, C, H, W) B might be less than B for last shard if total images are not perfectly divible into shards, syntax works
        
        # advance current position by one block (8*B in this case)
        self.current_position += self.B * self.num_processes
        # if loading the next batch would be out of bounds, reset
        if self.current_position + (B * self.num_processes) > len (self.tokens): # len (B, C, H, W) returns B
            
            if self.current_shard + 1 >= len(self.shards):
                self.current_epoch = self.current_epoch + 1
            self.current_shard = (self.current_shard + 1) % len(self.shards)
            
            self.tokens = load_tokens (self.shards[self.current_shard])
            self.current_position = self.process_rank * B
            # total len (tokens) = 9B
            # 0           1                                2                                3 4 5 6 7 
            # 8B to 9B    9(x)[shard2 1B to 2B instead]    10(x)[shard2 2B to 3B instead]
            # third gpu loads 2B-3B from next shard first two load remainder of the current shard
            # if remainder is not divisible by in the last shard [a:b] loads from a to c (c < b)
        return x

In [14]:
import numpy as np
train_loader = DataloaderLite (4, 1, 0, "train", "./dummy_tests/shards")

found 1 shards for split train


In [22]:
reference = train_loader.tokens[:4,:,:,:]

In [23]:
reference.shape

torch.Size([4, 3, 256, 256])

In [24]:
loader_batch = train_loader.next_batch()
loader_batch.shape

torch.Size([4, 3, 256, 256])

In [25]:
reference.allclose(loader_batch)

True

In [12]:
import os 
os.listdir("./dummy_tests/shards")

['shard_train_0001.npy', 'shard_val_0000.npy']

In [18]:
os.path.exists(train_loader.shards[train_loader.current_shard])

True

In [28]:
reference.dtype

torch.float32

In [29]:
reference[0,:,:,:]

tensor([[[-0.3647, -0.3647, -0.3647,  ..., -0.8037, -0.8115, -0.8115],
         [-0.3257, -0.3257, -0.3335,  ..., -0.7568, -0.7646, -0.7646],
         [-0.2939, -0.2939, -0.3018,  ..., -0.6943, -0.7021, -0.7021],
         ...,
         [ 0.0820,  0.0430,  0.0195,  ..., -0.3491, -0.3413, -0.3335],
         [ 0.0977,  0.0664,  0.0742,  ..., -0.4038, -0.3960, -0.3960],
         [ 0.0273,  0.0117,  0.0352,  ..., -0.4351, -0.4272, -0.4194]],

        [[-0.7021, -0.7021, -0.7021,  ..., -0.8589, -0.8667, -0.8667],
         [-0.6631, -0.6631, -0.6709,  ..., -0.8271, -0.8354, -0.8354],
         [-0.6313, -0.6313, -0.6392,  ..., -0.7803, -0.7881, -0.7881],
         ...,
         [-0.2705, -0.3096, -0.3335,  ..., -0.6470, -0.6392, -0.6313],
         [-0.2393, -0.2705, -0.2627,  ..., -0.7100, -0.7021, -0.7021],
         [-0.3096, -0.3257, -0.3018,  ..., -0.7412, -0.7334, -0.7256]],

        [[-0.6470, -0.6470, -0.6470,  ..., -0.9058, -0.9136, -0.9136],
         [-0.6079, -0.6079, -0.6157,  ..., -0

In [47]:
def load_tokens (filename):
    npt = np.load (filename) # shape (B, H, W, C) because PIL was used to store
    print(npt.dtype)
    ptt = torch.from_numpy (npt).permute (0, 3, 1, 2)
    print (ptt.dtype)
    ptt = ptt.to(torch.float32) # to match conv layers data type
    return ptt

In [48]:
a = load_tokens("./dummy_tests/shards/shard_train_0001.npy")

float32
torch.float32


In [33]:
a.dtype

torch.float32